# Getting Started with NVFlare (PyTorch)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/NVFlare/blob/main/examples/getting_started/pt/nvflare_pt_getting_started.ipynb)

NVFlare is an open-source framework that allows researchers and
data scientists to seamlessly move their machine learning and deep
learning workflows into a federated paradigm.

## Basic Concepts
At the heart of NVFlare lies the concept of collaboration through
"tasks." An FL controller assigns tasks (e.g., training on local data) to one or more FL clients, processes returned
results (e.g., model weight updates), and may assign additional
tasks based on these results and other factors (e.g., a pre-configured
number of training rounds). The clients run executors which can listen for tasks and perform the necessary computations locally, such as model training. This task-based interaction repeats
until the experiment’s objectives are met. 

<img src="../../../docs/resources/controller_executor_no_filter.png" alt="NVIDIA FLARE Controller and Executor" width=75% height=75% />

## Setup environment

Install nvflare and dependencies:

In [ ]:
! pip install --ignore-installed blinker
! pip install nvflare~=2.5.0rc torch torchvision tensorboard

If running in Google Colab, download the source code for this example:

In [ ]:
! npx degit NVIDIA/NVFlare/examples/getting_started/pt/src src

## Federated Averaging with NVFlare
Given the flexible controller and executor concepts, it is easy to implement different computing & communication patterns with NVFlare, such as [FedAvg](https://proceedings.mlr.press/v54/mcmahan17a?ref=https://githubhelp.com) and [cyclic weight transfer](https://academic.oup.com/jamia/article/25/8/945/4956468). 

The controller's `run()` routine is responsible for assigning tasks and processing task results from the Executors. 

### Server Code
First, we provide a simple implementation of the [FedAvg](https://proceedings.mlr.press/v54/mcmahan17a?ref=https://githubhelp.com) algorithm with NVFlare. 
The `run()` routine implements the main algorithmic logic. 
Subroutines, like `sample_clients()` and `scatter_and_gather_model()` utilize the communicator object, native to each Controller to get the list of available clients,
distribute the current global model to the clients, and collect their results.

The FedAvg controller implements these main steps:
1. FL server initializes an initial model using `self.load_model()`.
2. For each round (global iteration):
    - FL server samples available clients using `self.sample_clients()`.
    - FL server sends the global model to clients and waits for their updates using `self.send_model_and_wait()`.
    - FL server aggregates all the `results` and produces a new global model using `self.update_model()`.

```python
class FedAvg(BaseFedAvg):
    def run(self) -> None:
        self.info("Start FedAvg.")

        model = self.load_model()
        model.start_round = self.start_round
        model.total_rounds = self.num_rounds

        for self.current_round in range(self.start_round, self.start_round + self.num_rounds):
            self.info(f"Round {self.current_round} started.")
            model.current_round = self.current_round

            clients = self.sample_clients(self.num_clients)

            results = self.send_model_and_wait(targets=clients, data=model)

            aggregate_results = self.aggregate(results)

            model = self.update_model(model, aggregate_results)

            self.save_model(model)

        self.info("Finished FedAvg.")
```

### Client Code 
We take a CIFAR-10 example directly from [PyTorch website](https://github.com/pytorch/tutorials/blob/main/beginner_source/blitz/cifar10_tutorial.py) with some minor modifications, such as removing comments, move the network to [src/net.py](src/net.py), and add a main method and GPU support. The original code can be found at [cifar10_original.py](../../hello-world/ml-to-fl/pt/code/cifar10_original.py).

Now, we need to adapt this centralized training code to something that can run in a federated setting.

On the client side, the training workflow is as follows:
1. Receive the model from the FL server.
2. Perform local training on the received global model
and/or evaluate the received global model for model
selection.
3. Send the new model back to the FL server.

Using NVFlare's client API, we can easily adapt machine learning code that was written for centralized training and apply it in a federated scenario.
For a general use case, there are three essential methods to achieve this using the Client API :
- `init()`: Initializes NVFlare Client API environment.
- `receive()`: Receives model from the FL server.
- `send()`: Sends the model to the FL server.

With these simple methods, the developers can use the Client API
to change their centralized training code to an FL scenario with
five lines of code changes as shown below.
```python
    import nvflare.client as flare
    
    flare.init() # 1. Initializes NVFlare Client API environment.
    input_model = flare.receive() # 2. Receives model from the FL server.
    params = input_model.params # 3. Obtain the required information from the received model.
    
    # original local training code
    new_params = local_train(params)
    
    output_model = flare.FLModel(params=new_params) # 4. Put the results in a new `FLModel`
    flare.send(output_model) # 5. Sends the model to the FL server.  
```

The full client training script is saved in a separate file, e.g. [./src/cifar10_fl.py](./src/cifar10_fl.py) doing CNN training on the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset.

## Run an NVFlare Job
Now that we have defined the FedAvg controller to run our federated compute workflow on the FL server, and our client training script to receive the global models, run local training, and send the results back to the FL server, we can put everything together using NVFlare's Job API.

#### 1. Define the initial model
First, we define the global model used to initialize the model on the FL server. See [src/net.py](src/net.py).

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
```

#### 2. Define a FedJob
The `FedJob` is used to define how controllers and executors are placed within a federated job using the `to(object, target)` routine.

In [12]:
from nvflare import FedJob
from nvflare.app_common.executors.script_executor import ScriptExecutor
from nvflare.app_common.workflows.fedavg import FedAvg

job = FedJob(name="cifar10_fedavg")

#### 3. Define the Controller Workflow
Define the controller workflow and send to server.

In [13]:
n_clients = 2

controller = FedAvg(
    num_clients=n_clients,
    num_rounds=2,
)
job.to(controller, "server")

#### 4. Create Global Model
Now, we create the initial global model and send to server.

In [14]:
from src.net import Net
from nvflare.job_config.pt.model import PTModel

job.to(PTModel(Net()), "server")

That completes the components that need to be defined on the server.

#### 5. Add clients
Next, we can use the `ScriptExecutor` and send it to each of the clients to run our training script.

Note that our script could have additional input arguments, such as batch size or data path, but we don't use them here for simplicity.
We can also specify, which GPU should be used to run this client, which is helpful for simulated environments.

In [15]:
for i in range(n_clients):
    executor = ScriptExecutor(
        task_script_path="src/cifar10_fl.py", task_script_args=""  # f"--batch_size 32 --data_path /tmp/data/site-{i}"
    )
    job.to(executor, f"site-{i+1}")

That's it!

#### 6. Optionally export the job
Now, we could export the job and submit it to a real NVFlare deployment using the [Admin client](https://nvflare.readthedocs.io/en/main/real_world_fl/operation.html) or [FLARE API](https://nvflare.readthedocs.io/en/main/real_world_fl/flare_api.html).

In [16]:
job.export_job("/tmp/nvflare/jobs/job_config")

#### 7. Run FL Simulation
Finally, we can run our FedJob in simulation using NVFlare's [simulator](https://nvflare.readthedocs.io/en/main/user_guide/nvflare_cli/fl_simulator.html) under the hood. The results will be saved in the specified `workdir`.

In [17]:
job.simulator_run("/tmp/nvflare/jobs/workdir", gpu="0")

2024-08-16 12:29:21,325 - SimulatorRunner - INFO - When running with multi GPU, each GPU group will run with only 1 thread. Set the Threads to 1.
2024-08-16 12:29:21,327 - SimulatorRunner - INFO - Create the Simulator Server.
2024-08-16 12:29:21,329 - CoreCell - INFO - server: creating listener on tcp://0:34219
2024-08-16 12:29:21,338 - CoreCell - INFO - server: created backbone external listener for tcp://0:34219
2024-08-16 12:29:21,339 - ConnectorManager - INFO - 27586: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2024-08-16 12:29:21,340 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:26151] is starting
2024-08-16 12:29:21,841 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:26151
2024-08-16 12:29:21,842 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:34219] is starting
2024-08-16 12:29:21,908 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server loc

2024-08-16 12:29:26.813808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-08-16 12:29:26.847750: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2024-08-16 12:29:28,017 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connection [CN00005 127.0.0.1:34219 <= 127.0.0.1:50372] is created: PID: 27586
2024-08-16 12:29:28,030 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connection [CN00006 127.0.0.1:34219 <= 127.0.0.1:50382] is created: PID: 27586
2024-08-16 12:29:27,967 - ClientTaskWorker - INFO - ClientTaskWorker started to run
2024-08-16 12:29:27,981 - ClientTaskWorker - INFO - ClientTaskWorker started to run
2024-08-16 12:29:28,016 - CoreCell - INFO - site-1.simulate_job: created backbone external connector to tcp://localhost:34219
2024-08-16 12:29:28,016 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 ACTIVE tcp://localhost:34219] is starting
2024-08-16 12:29:28,017 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connection [CN00002 127.0.0.1:50372 => 127.0.0.1:34219] is created: PID: 27698
2024-08-16 12:29:28,029 - CoreCell - INFO - site-2.simulate_job: created backbone external connector to tcp://localhost:34219
2024-08-16

If using Google Colab and the output is not showing correctly, export the job and run it with the simulator command instead:

In [ ]:
! nvflare simulator -w /tmp/nvflare/jobs/workdir -n 2 -t 2 -gpu 0 /tmp/nvflare/jobs/job_config/cifar10_fedavg

tensorboard --logdir=/tmp/nvflare/jobs/workdir/server/simulate_job/tb_events
tensorboard --logdir=/tmp/nvflare/jobs/workdir/server/simulate_job/tb_events### Visualize the Training Results
By default, we enable TensorBoard metric [streaming](https://nvflare.readthedocs.io/en/main/examples/tensorboard_streaming.html) using NVFlare's `SummaryWriter` in [src/cifar10_fl.py](src/cifar10_fl.py). 

The TensorBoard metrics will be received at the server, and you can visualize the training progress by running 
```commandline
tensorboard --logdir=/tmp/nvflare/jobs/workdir/server/simulate_job/tb_events
```
in a new terminal.

<img src="figs/tb_loss.png" alt="Streamed TensorBoard metric" width=50% height=50% />